In [ ]:
%pylab inline
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sherpa.astro import ui

In [ ]:
ui.load_pha('3c273.pi')

In [ ]:
ui.plot_data()

In [ ]:
ui.notice(0.1, 6.0)

In [ ]:
ui.subtract()

In [ ]:
ui.plot_data()

In [ ]:
ui.set_xlog()
ui.set_ylog()
ui.plot_data()

In [ ]:
ui.list_models()

In [ ]:
from sherpa.astro import *
from sherpa.models import *
#from sherpa.models.astro.xspec import get_xsversion
#from sherpa.models import xspec
#ui.set_source(PowLaw1d.p1)

In [ ]:
truth = PowLaw1D()

In [ ]:
#ui.guess(truth.p1)

### Let's try the image
Fitting an image using a PSF image

In [ ]:
ui.load_image("center_box_0.25pix.fits")

In [ ]:
#ui.image_getregion()
#ui.notice2d("rotbox(88.16875,75.8625,70.416667,68.508334,0)")

In [ ]:
ui.load_psf("psf0","psf_f1_norm_0.25pix.fits")

In [ ]:
ui.set_psf("psf0")

In [ ]:
psf0.center = [128,129]
psf0.size = [72,72]
ui.set_psf(psf0)

In [ ]:
print(ui.get_psf())

In [ ]:
# We now set a Gaussian source model with a constant background and thaw ellipticity 
# and rotation angle. We also set the limits for background
ui.set_source(ui.gauss2d.g1)

In [ ]:
g1.fwhm = 6
g1.xpos = 90
g1.ypos = 80
g1.ampl = 100

In [ ]:
# Here we set the statistic to the Cash statisitc, and the optimization method to Simplex.
ui.set_stat('cstat')
ui.set_method('simplex')

In [ ]:
# Let us fit, by finding a MAP estimate. First, we have sherpa guessing initial values
ui.guess(g1)
ui.fit()

In [ ]:
# Let us fit again, just in case the optimizer got stuck
ui.fit()

In [ ]:
# We now print the model
print(ui.get_source())

In [ ]:
# Let us add background
ui.set_source(g1 + ui.const2d.c1)
c1.c0 = 1.0

In [ ]:
ui.freeze(c1.c0)

In [ ]:
print(ui.get_source())

In [ ]:
# Fit
ui.fit()

In [ ]:
# Do it again
ui.fit()

The fit is run again with the background component thawed:

In [ ]:
ui.thaw(c1.c0)
ui.fit() 

Now, we will get the confidence level of the fit, and in doing so, recalculate the best-fit parameters using the confidence method.

In [ ]:
ui.conf()

In [ ]:
# Let us get a covariance matrix
ui.covar()

In [ ]:
# Now we run MCMCM
draws = ui.get_draws(niter=5000)

In [ ]:
svals, accept, pvals = draws

In [ ]:
# Acceptance rate
accept.sum() * 1.0 / 5000

In [ ]:
import matplotlib.pyplot as plt
# Now let us plot the traces
plt.subplots(3,2,figsize=(10,10))
plt.subplot(3,2,1)
plt.plot(pvals[0, :])
plt.xlabel('Iteration')
plt.ylabel('fwhm')
plt.subplot(3,2,2)
plt.plot(pvals[1, :])
plt.xlabel('Iteration')
plt.ylabel('xpos')
plt.subplot(3,2,3)
plt.plot(pvals[2, :])
plt.xlabel('Iteration')
plt.ylabel('ypos')
plt.subplot(3,2,4)
plt.plot(pvals[3, :])
plt.xlabel('Iteration')
plt.ylabel('ampl')
plt.subplot(3,2,5)
plt.plot(pvals[4, :])
plt.xlabel('Iteration')
plt.ylabel('bkg')

In [ ]:
from sherpa import plot
pdf = plot.PDFPlot()
pdf.prepare(pvals[0, :], 20, False, 'fwhm', name='fwhm')
pdf.plot()

In [ ]:
pdf.prepare(pvals[1, :], 20, False, 'xpos', name='xpos')
pdf.plot()

In [ ]:
pdf.prepare(pvals[2, :], 20, False, 'ypos', name='ypos')
pdf.plot()

In [ ]:
pdf.prepare(pvals[3, :], 20, False, 'ampl', name='ampl')
pdf.plot()

In [ ]:
pdf.prepare(pvals[4, :], 20, False, 'bkg', name='bkg')
pdf.plot()